<a href="https://colab.research.google.com/github/ShravyaMalogi/PROJECT_drafts/blob/main/seniorcitizen/age_gender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 👵 Senior Citizen Identifier - Multitask Model Training (Age + Gender)

In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
jangedoo_utkface_new_path = kagglehub.dataset_download('jangedoo/utkface-new')

print('Data source import complete.')

100%|██████████| 331M/331M [00:16<00:00, 21.2MB/s]

Extracting files...


Data source import complete.


In [4]:
# Assuming UTKFace is extracted under the downloaded kaggle directory
dataset_path = os.path.join(jangedoo_utkface_new_path, 'UTKFace')
image_paths = [os.path.join(dataset_path, img) for img in os.listdir(dataset_path) if img.endswith('.jpg')]

images = []
ages = []
genders = []

for path in tqdm(image_paths[:10000]):  # take subset to reduce memory issues
    try:
        filename = os.path.basename(path)
        age, gender, _ = filename.split('_')[:3]
        age = int(age)
        gender = int(gender)

        img = Image.open(path).convert('RGB')
        img = img.resize((128, 128))
        img = np.asarray(img).astype('float32') / 255.0

        images.append(img)
        ages.append(age / 100)  # normalized
        genders.append(gender)
    except:
        continue

X = np.array(images)
y_age = np.array(ages)
y_gender = np.array(genders)

100%|██████████| 10000/10000 [00:13<00:00, 754.16it/s]


In [5]:
X_train, X_test, y_age_train, y_age_test, y_gender_train, y_gender_test = train_test_split(
    X, y_age, y_gender, test_size=0.2, random_state=42)

In [6]:

from keras.applications import MobileNetV2
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model

base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(128, 128, 3))
base_model.trainable = False  # freeze to speed up training

x = base_model.output
x = GlobalAveragePooling2D()(x)

# Age regression head
age_output = Dense(1, activation='linear', name='age_output')(x)

# Gender classification head
gender_output = Dense(1, activation='sigmoid', name='gender_output')(x)

model = Model(inputs=base_model.input, outputs=[age_output, gender_output])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 64, 64,    │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 64, 64,    │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 64, 64,    │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 64, 64,    │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 64, 64,    │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 64, 64,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 65, 65,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 32, 32,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 32, 32,    │      2,304 │ block_1_depthwis

 Total params: 2,260,546 (8.62 MB)

 Trainable params: 2,562 (10.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [7]:
checkpoint = ModelCheckpoint(
    "/content/drive/MyDrive/best_age_gender_model.keras",
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [8]:

from keras.optimizers import Adam

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss={
        'age_output': 'mse',
        'gender_output': 'binary_crossentropy'
    },
    metrics={
        'age_output': ['mae'],
        'gender_output': ['accuracy']
    }
)


In [9]:

from keras.callbacks import ModelCheckpoint, EarlyStopping

history = model.fit(
    X_train,
    {'age_output': y_age_train, 'gender_output': y_gender_train},
    validation_data=(X_test, {'age_output': y_age_test, 'gender_output': y_gender_test}),
    epochs=25,
    batch_size=32,
    callbacks=[checkpoint, early_stop]
)


Epoch 1/25
249/250 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - age_output_loss: 0.7296 - age_output_mae: 0.6688 - gender_output_accuracy: 0.5769 - gender_output_loss: 0.6799 - loss: 1.4095
Epoch 1: val_loss improved from inf to 0.88197, saving model to /content/drive/MyDrive/best_age_gender_model.keras
250/250 ━━━━━━━━━━━━━━━━━━━━ 25s 58ms/step - age_output_loss: 0.7278 - age_output_mae: 0.6679 - gender_output_accuracy: 0.5774 - gender_output_loss: 0.6795 - loss: 1.4073 - val_age_output_loss: 0.3333 - val_age_output_mae: 0.4601 - val_gender_output_accuracy: 0.7335 - val_gender_output_loss: 0.5468 - val_loss: 0.8820
Epoch 2/25
247/250 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - age_output_loss: 0.2944 - age_output_mae: 0.4270 - gender_output_accuracy: 0.7238 - gender_output_loss: 0.5465 - loss: 0.8410
Epoch 2: val_loss improved from 0.88197 to 0.72664, saving model to /content/drive/MyDrive/best_age_gender_model.keras
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - age_output_loss: 0.2940 - age_output_mae:

In [12]:
from keras.models import load_model

model = load_model("/content/drive/MyDrive/best_age_gender_model.keras")


In [17]:
history = model.fit(
    X_train,
    {'age_output': y_age_train, 'gender_output': y_gender_train},
    validation_data=(X_test, {'age_output': y_age_test, 'gender_output': y_gender_test}),
    initial_epoch=80,      # Start from epoch 26
    epochs=100,             # Go till epoch 40 (or whatever target)
    batch_size=32,
    callbacks=[checkpoint, early_stop]  # optional
)

Epoch 81/100
246/250 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - age_output_loss: 0.0120 - age_output_mae: 0.0852 - gender_output_accuracy: 0.8617 - gender_output_loss: 0.3293 - loss: 0.3413
Epoch 81: val_loss did not improve from 0.38565
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - age_output_loss: 0.0120 - age_output_mae: 0.0852 - gender_output_accuracy: 0.8617 - gender_output_loss: 0.3292 - loss: 0.3412 - val_age_output_loss: 0.0187 - val_age_output_mae: 0.1056 - val_gender_output_accuracy: 0.8330 - val_gender_output_loss: 0.3661 - val_loss: 0.3862
Epoch 82/100
249/250 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - age_output_loss: 0.0121 - age_output_mae: 0.0850 - gender_output_accuracy: 0.8543 - gender_output_loss: 0.3331 - loss: 0.3453
Epoch 82: val_loss improved from 0.38565 to 0.38558, saving model to /content/drive/MyDrive/best_age_gender_model.keras
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - age_output_loss: 0.0121 - age_output_mae: 0.0850 - gender_output_accuracy: 0.8544 - gender_output_loss: 

# New section

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
save_path = '/content/drive/MyDrive/senior_citizen_project/best_age_model.keras'

In [ ]:
from google.colab import files
model.save("age_gender_model.keras")

# Download to your local system
files.download("age_gender_model.keras")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
# Predict on test set
age_pred_norm, _ = model.predict(X_test)

# Denormalize predictions and ground truth
age_pred = age_pred_norm.flatten() * 100
age_true = y_age_test * 100

# Compute MAE in years
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(age_true, age_pred)

print(f"De-normalized MAE: {mae:.2f} years")

63/63 ━━━━━━━━━━━━━━━━━━━━ 9s 54ms/step
De-normalized MAE: 10.09 years
